## This explores the variability of AMOC time series.

In [2]:
# Preprocessing, and import data here


In [ ]:
import pandas as pd

# Load your data (adjust path and separator as needed)
df = pd.read_csv("AMOC_test.txt", sep="\t")

# Drop the 'year' column and isolate the AMOC values
data = df.drop(columns='year')

# Step 1: Compute anomalies (remove mean from each column)
anomalies = data.apply(lambda x: x - x.mean())

# Step 2: Interannual variability (variance of anomalies)
interannual_var = anomalies.var()

# Step 3: Decadal variability
# Smooth with a 10-year rolling mean and compute variance again
decadal_smoothed = anomalies.rolling(window=10, center=True).mean()
decadal_var = decadal_smoothed.var()

# Combine results in a table
variability_comparison = pd.DataFrame({
    'Interannual Variability (var)': interannual_var,
    'Decadal Variability (var)': decadal_var
})

# Display result
print(variability_comparison)